In [37]:
import os
import requests
import tweepy as tw
import pandas as pd
from datetime import datetime
import configparser
from google_trans_new import google_translator

In [2]:
# API set-ups for the use of Twitter API
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

auth = tw.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth) 

In [15]:
### Search var. 1.2
# Container to collect data
search_results_dict = {}
# Searching function. !only tweets from first twitter page will be returned
def search_tweets(search_query, no_of_tweets=5):
    try:
        #Search without retweets
        tweets = api.search_tweets(q=search_query+' -filter:retweets', result_type='recent', count=no_of_tweets, tweet_mode='extended')       
        #Pulling Some attributes from the tweet
        attributes_container = [[tweet.user.screen_name, tweet.created_at, tweet.full_text, tweet.lang] for tweet in tweets]
        #Collecting results to dict
        search_results_dict[search_query] = attributes_container
    except BaseException as e:
        print('Status Failed On,',str(e))
    return search_results_dict

In [20]:
# !!!! CHECK sheet name !!!!!!!!!
# Getting search list from excel file
path = 'D:\\sorare.xlsx'
search_df = pd.read_excel(path, sheet_name='search_short', header=0, usecols="B")
search_list = search_df['Search Name'].tolist()
for query in search_list:
     search_tweets(query, 5)


In [21]:
## Search result to dataframe
#Creation of column list to rename the columns in the dataframe
columns = ["User", "Date Created", "Tweet", "Language"]
#Creation of Dataframe
tweets_df = pd.concat({k: pd.DataFrame(v) for k, v in search_results_dict.items()}, axis=0)
tweets_df.columns = columns
tweets_df.head()

User              Date Created  \
Lucas Digne transfer     0     Utd_Bey 2022-10-17 13:20:13+00:00   
                         1      BTLvid 2022-10-15 14:29:24+00:00   
Jairo Torres transfer    0  SorareDave 2022-10-18 08:52:10+00:00   
Juan Agudelo Birmingham  0     abc3340 2022-10-14 23:55:00+00:00   
                         1     abc3340 2022-10-14 00:34:00+00:00   

                                                                        Tweet  \
Lucas Digne transfer     0  @FergiesRightRef @markgoldbridge @UnitedStandM...   
                         1  Aston Villa’s loan deals which saw Philippe Co...   
Jairo Torres transfer    0  2 quick examples of where I've failed to take ...   
Juan Agudelo Birmingham  0  The wait lasted longer than many Birmingham fa...   
                         1  Birmingham Legion's home playoff game is now s...   

                           Language  
Lucas Digne transfer     0       en  
                         1       en  
Jairo Torres transfer    0       en  
Juan Agudelo Birmingham  0       en  
                         1       en

In [36]:
none_eng_tw = tweets_df[tweets_df['Language'] != 'en']#['Language'].tolist()
# none_eng_tw['Tweet'].shape[0]
for i in range(none_eng_tw['Tweet'].shape[0]):
    print(len(none_eng_tw['Tweet'].iloc[i].encode("utf8")))
# Translate tweets from spanish, french etc

# https://api.mymemory.translated.net/get?q=Hello%20World!&langpair=en|it

# translator = google_translator()
# none_eng_tw['Translated'] = none_eng_tw['Tweet'].apply(lambda x: translator.translate(x, lang_tgt ='en'))
# none_eng_tw.head()

261
294
81
126
136
93
183
44
64
61
109
303
291
281
273
37
43
105
284
146
185
305


In [22]:
# Data upload to excel
tweets_df.reset_index(level=[0,1], inplace=True)
tweets_df.drop('level_1', axis=1,inplace=True)
tweets_df['Date Created'] = tweets_df['Date Created'].dt.tz_localize(None)
today = datetime.now().strftime("%d.%m.%Y_%H-%M")
file_name = 'tweets ' + today + '.xlsx'
tweets_df.to_excel(file_name)

In [8]:
# # var. 2 - for multi page
# def get_tweets(list_of_tweets, keyword, num_of_tweets, data_until):
#     for tweet in tw.Cursor(api.search_tweets, q=keyword+' -filter:retweets', until=data_until, lang='en').items(num_of_tweets):
#         dict_ = {
#                  'Keywords': keyword,
#                  'User Name': tweet.user.name,
#                  'Screen Name': tweet.user.screen_name,
#                  'Tweet Created at': tweet.created_at,
#                  'Tweet Text': tweet.text,
#                  'Location': tweet.user.location,
#                  'Likes': tweet.favorite_count,
#                  'Retweets': tweet.retweet_count
#                  }
#         list_of_tweets.append(dict_)
#     return list_of_tweets